# Training Notebook

## Installation

In [1]:
!git clone https://github.com/JerBaf/OptML_project

Cloning into 'OptML_project'...


In [ ]:
!cp /kaggle/working/OptML_project/model.py /kaggle/working/model.py
!cp /kaggle/working/OptML_project/nb_optimizers.py /kaggle/working/nb_optimizers.py

## Imports

In [1]:
import torch
import torchvision
import ssl
import random
import numpy as np
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import tqdm.notebook as tqdm
from torch.utils.data.sampler import SubsetRandomSampler
import os
# Custom helpers
from model import *
import nb_optimizers as opt
# Allow autoreload
%load_ext autoreload
%autoreload 2
# Allow Download of CIFAR datasets
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = "cpu"
print(device)
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

cuda:0
Setup complete. Using torch 1.10.0+cu113 _CudaDeviceProperties(name='NVIDIA GeForce RTX 2060', major=7, minor=5, total_memory=6143MB, multi_processor_count=30)


## Helpers

In [3]:
### Helpers
def seed_worker(worker_id):
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

def get_samplers(train_dataset,generator,shuffle=True,val_ratio=0.1):
    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(val_ratio * num_train))
    if shuffle:
        np.random.shuffle(indices)
    train_idx, val_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx,generator=generator)
    val_sampler = SubsetRandomSampler(val_idx,generator=generator)
    return train_sampler, val_sampler
  
def test_step(model,test_dataloader,device,criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in tqdm.tqdm(test_dataloader,leave=False):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    accuracy = 100.*correct/total
    return accuracy

## Testing Pipeline

In [4]:
def testing_pipeline(dataset,model_path,seed=2022,batch_size=1024):
    ### Reproducibility
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    g = torch.Generator()
    g.manual_seed(seed)
    ### Download Datasets
    if dataset == "CIFAR10":
        dataset_train = torchvision.datasets.CIFAR10("data/",download=True)
        dataset_test = torchvision.datasets.CIFAR10("data/",download=True,train=False)
    elif dataset == "CIFAR100":
        dataset_train = torchvision.datasets.CIFAR100("data/",download=True)
        dataset_test = torchvision.datasets.CIFAR100("data/",download=True,train=False)
    elif dataset == "MNIST":
        dataset_train = torchvision.datasets.MNIST("data/",download=True)
        dataset_test = torchvision.datasets.MNIST("data/",download=True,train=False)
    elif dataset == "FashionMNIST":
        dataset_train = torchvision.datasets.FashionMNIST("data/",download=True)
        dataset_test = torchvision.datasets.FashionMNIST("data/",download=True,train=False)
    else:
        raise Exception("Unavailable dataset, please select among CIFAR10, CIFAR100, MNIST, FashionMNIST.")
      ### Compute initial Transform
    if dataset in ["CIFAR10","CIFAR100"]:
        mean_per_channel = tuple((dataset_train.data/255).mean(axis=(0,1,2)))
        std_per_channel = tuple((dataset_train.data/255).std(axis=(0,1,2)))
    else:
        mean_per_channel = (dataset_train.data.numpy()/255).mean()
        std_per_channel = (dataset_train.data.numpy()/255).std()

    base_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean_per_channel, std_per_channel),
      ])
    noise_transform = transforms.Compose([
        transforms.GaussianBlur(3),
        transforms.ToTensor(),
        transforms.Normalize(mean_per_channel, std_per_channel),
      ])
    affine_transform = transforms.Compose([
        transforms.RandomAffine(20,translate=(0.2,0.2),scale=(0.85,1.15)),
        transforms.ToTensor(),
        transforms.Normalize(mean_per_channel, std_per_channel),
      ])
    flip_transform = transforms.Compose([
        transforms.RandomVerticalFlip(0.5),
        transforms.RandomHorizontalFlip(0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean_per_channel, std_per_channel),
      ])
  ### Dataset Creation
    if dataset == "CIFAR10":
        dataset_test = torchvision.datasets.CIFAR10("data/",transform=base_transform,train=False)
        dataset_noise = torchvision.datasets.CIFAR10("data/",transform=noise_transform,train=False)
        dataset_affine = torchvision.datasets.CIFAR10("data/",transform=affine_transform,train=False)
        dataset_flip = torchvision.datasets.CIFAR10("data/",transform=flip_transform,train=False)
    elif dataset == "CIFAR100":
        dataset_test = torchvision.datasets.CIFAR100("data/",transform=base_transform,train=False)
        dataset_noise = torchvision.datasets.CIFAR100("data/",transform=noise_transform,train=False)
        dataset_affine = torchvision.datasets.CIFAR100("data/",transform=affine_transform,train=False)
        dataset_flip = torchvision.datasets.CIFAR100("data/",transform=flip_transform,train=False)
    elif dataset == "MNIST":
        dataset_test = torchvision.datasets.MNIST("data/",transform=base_transform,train=False)
        dataset_noise = torchvision.datasets.MNIST("data/",transform=noise_transform,train=False)
        dataset_affine = torchvision.datasets.MNIST("data/",transform=affine_transform,train=False)
        dataset_flip = torchvision.datasets.MNIST("data/",transform=flip_transform,train=False)
    elif dataset == "FashionMNIST":
        dataset_test = torchvision.datasets.FashionMNIST("data/",transform=base_transform,train=False)
        dataset_noise = torchvision.datasets.FashionMNIST("data/",transform=noise_transform,train=False)
        dataset_affine = torchvision.datasets.FashionMNIST("data/",transform=affine_transform,train=False)
        dataset_flip = torchvision.datasets.FashionMNIST("data/",transform=flip_transform,train=False)
    ### Dataloaders creation
    dataloader_test = DataLoader(dataset_test,batch_size=batch_size,pin_memory=True,
                                  worker_init_fn=seed_worker, generator=g, shuffle=True)
    dataloader_noise = DataLoader(dataset_noise,batch_size=batch_size,pin_memory=True,
                                  worker_init_fn=seed_worker, generator=g, shuffle=True)
    dataloader_affine = DataLoader(dataset_affine,batch_size=batch_size,pin_memory=True,
                                  worker_init_fn=seed_worker, generator=g, shuffle=True)
    dataloader_flip = DataLoader(dataset_flip,batch_size=batch_size,pin_memory=True,
                                  worker_init_fn=seed_worker, generator=g, shuffle=True)

      ### Model Creation
    if dataset == "CIFAR10":
        in_c, out_c = (3,10)
    elif dataset == "CIFAR100":
        in_c, out_c = (3,100)
    elif dataset == "MNIST":
        in_c, out_c = (1,10)
    elif dataset == "FashionMNIST":
        in_c, out_c = (1,10)
    model = VGG(in_c,out_c)
    init_checkpoint = torch.load(model_path)
    state_dict = init_checkpoint["model"]
    formated_state_dict = dict()
    for key, value in state_dict.items():
        formated_state_dict[key.split("module.")[-1]] = value
    model.load_state_dict(formated_state_dict)
    model.to(device)
    if device.type == 'cuda':
        model = torch.nn.DataParallel(model)
        cudnn.benchmark = True
    ### Old Validation Accuracy
    validation_accuracy = init_checkpoint["accuracy"]
    print(f"\nThe model has a validation accuracy of {validation_accuracy:.5f}%")
    ### Criterion Definition
    criterion = nn.CrossEntropyLoss()
    ### Testing steps
    base_acc = test_step(model,dataloader_test,device,criterion)
    noise_acc = test_step(model,dataloader_noise,device,criterion)
    affine_acc = test_step(model,dataloader_affine,device,criterion)
    flip_acc = test_step(model,dataloader_flip,device,criterion)
    print("The model has the following perfomances:")
    print(f"-\tAccuracy on the standard test set: {base_acc:.5f}%")
    print(f"-\tAccuracy on the noisy test set: {noise_acc:.5f}%")
    print(f"-\tAccuracy on the affine transformed test set: {affine_acc:.5f}%")
    print(f"-\tAccuracy on the flipped test set: {flip_acc:.5f}%")
    return base_acc, noise_acc, affine_acc, flip_acc

## Runs

### SGD

##### CIFAR10

In [6]:
base_acc, noise_acc, affine_acc, flip_acc = testing_pipeline("CIFAR10","../../project/bests/CIFAR10_SGD/CIFAR10_sgd_custom_lr_3e-1_rho_9e-1_ckpt.pth")

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data/
Files already downloaded and verified

The model has a validation accuracy of 84.00000%


  0%|          | 0/10 [00:00<?, ?it/s]

C:\EPFL\master\MA_2\OptML\OptML_project\notebook\model.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = softmax_output(out)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

The model has the following perfomances:
-	Accuracy on the standard test set: 83.02000%
-	Accuracy on the noisy test set: 67.86000%
-	Accuracy on the affine transformed test set: 53.04000%
-	Accuracy on the flipped test set: 58.81000%


##### CIFAR100

In [7]:
base_acc, noise_acc, affine_acc, flip_acc = testing_pipeline("CIFAR100","../../project/bests/CIFAR100_SGD/ckpt.pth")

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data/
Files already downloaded and verified

The model has a validation accuracy of 43.58000%


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

The model has the following perfomances:
-	Accuracy on the standard test set: 42.69000%
-	Accuracy on the noisy test set: 32.33000%
-	Accuracy on the affine transformed test set: 18.51000%
-	Accuracy on the flipped test set: 28.30000%


##### FashionMNIST

In [8]:
base_acc, noise_acc, affine_acc, flip_acc = testing_pipeline("FashionMNIST","../../project/bests/FashionMNIST_SGD/ckpt.pth")

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST\raw\train-images-idx3-ubyte.gz to data/FashionMNIST\raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST\raw\train-labels-idx1-ubyte.gz to data/FashionMNIST\raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

The model has the following perfomances:
-	Accuracy on the standard test set: 93.33000%
-	Accuracy on the noisy test set: 87.34000%
-	Accuracy on the affine transformed test set: 47.18000%
-	Accuracy on the flipped test set: 54.01000%


##### MNIST

In [9]:
base_acc, noise_acc, affine_acc, flip_acc = testing_pipeline("MNIST","../../project/bests/MNIST_SGD/ckpt.pth")

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST\raw\train-images-idx3-ubyte.gz to data/MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST\raw\train-labels-idx1-ubyte.gz to data/MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

The model has the following perfomances:
-	Accuracy on the standard test set: 99.47000%
-	Accuracy on the noisy test set: 96.04000%
-	Accuracy on the affine transformed test set: 72.20000%
-	Accuracy on the flipped test set: 56.29000%


### MSGD

##### CIFAR10

In [10]:
base_acc, noise_acc, affine_acc, flip_acc = testing_pipeline("CIFAR10","../../project/bests/CIFAR10_MSGD/CIFAR10_msgd_custom_lr_2e-1_rho_9e-1_ckpt.pth")

Files already downloaded and verified
Files already downloaded and verified

The model has a validation accuracy of 86.06000%


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

The model has the following perfomances:
-	Accuracy on the standard test set: 84.89000%
-	Accuracy on the noisy test set: 58.56000%
-	Accuracy on the affine transformed test set: 56.61000%
-	Accuracy on the flipped test set: 60.56000%


##### CIFAR100

In [6]:
base_acc, noise_acc, affine_acc, flip_acc = testing_pipeline("CIFAR100","../../project/bests/CIFAR100_MSGD/CIFAR100_msgd_custom_lr_1e-1_rho_9e-1_ckpt.pth")

Files already downloaded and verified
Files already downloaded and verified

The model has a validation accuracy of 52.30000%


  0%|          | 0/10 [00:00<?, ?it/s]

C:\EPFL\master\MA_2\OptML\OptML_project\notebook\model.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = softmax_output(out)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

The model has the following perfomances:
-	Accuracy on the standard test set: 51.73000%
-	Accuracy on the noisy test set: 33.22000%
-	Accuracy on the affine transformed test set: 22.44000%
-	Accuracy on the flipped test set: 35.18000%


##### FashionMNIST

In [12]:
base_acc, noise_acc, affine_acc, flip_acc = testing_pipeline("FashionMNIST","../../project/bests/FashionMNIST_MSGD/ckpt.pth")


The model has a validation accuracy of 93.81667%


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

The model has the following perfomances:
-	Accuracy on the standard test set: 93.12000%
-	Accuracy on the noisy test set: 86.16000%
-	Accuracy on the affine transformed test set: 46.48000%
-	Accuracy on the flipped test set: 51.43000%


##### MNIST

In [13]:
base_acc, noise_acc, affine_acc, flip_acc = testing_pipeline("MNIST","../../project/bests/MNIST_MSGD/ckpt.pth")


The model has a validation accuracy of 99.45000%


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

The model has the following perfomances:
-	Accuracy on the standard test set: 99.57000%
-	Accuracy on the noisy test set: 98.57000%
-	Accuracy on the affine transformed test set: 74.61000%
-	Accuracy on the flipped test set: 56.46000%


### ADAM

##### CIFAR10

In [14]:
base_acc, noise_acc, affine_acc, flip_acc = testing_pipeline("CIFAR10","../../project/bests/CIFAR10_ADAM/ckpt.pth")

Files already downloaded and verified
Files already downloaded and verified

The model has a validation accuracy of 85.36000%


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

The model has the following perfomances:
-	Accuracy on the standard test set: 84.44000%
-	Accuracy on the noisy test set: 61.67000%
-	Accuracy on the affine transformed test set: 56.33000%
-	Accuracy on the flipped test set: 59.79000%


##### CIFAR100

In [15]:
base_acc, noise_acc, affine_acc, flip_acc = testing_pipeline("CIFAR100","../../project/bests/CIFAR100_ADAM/ckpt.pth")

Files already downloaded and verified
Files already downloaded and verified

The model has a validation accuracy of 40.08000%


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

The model has the following perfomances:
-	Accuracy on the standard test set: 38.14000%
-	Accuracy on the noisy test set: 29.28000%
-	Accuracy on the affine transformed test set: 14.36000%
-	Accuracy on the flipped test set: 25.84000%


##### FashionMNIST

In [16]:
base_acc, noise_acc, affine_acc, flip_acc = testing_pipeline("FashionMNIST","../../project/bests/FashionMNIST_ADAM/ckpt.pth")


The model has a validation accuracy of 93.53333%


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

The model has the following perfomances:
-	Accuracy on the standard test set: 92.57000%
-	Accuracy on the noisy test set: 86.17000%
-	Accuracy on the affine transformed test set: 49.05000%
-	Accuracy on the flipped test set: 53.92000%


##### MNIST

In [17]:
base_acc, noise_acc, affine_acc, flip_acc = testing_pipeline("MNIST","../../project/bests/MNIST_ADAM/ckpt.pth")


The model has a validation accuracy of 99.33333%


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

The model has the following perfomances:
-	Accuracy on the standard test set: 99.30000%
-	Accuracy on the noisy test set: 98.96000%
-	Accuracy on the affine transformed test set: 79.80000%
-	Accuracy on the flipped test set: 57.46000%


### RMSProp

##### CIFAR10

In [18]:
base_acc, noise_acc, affine_acc, flip_acc = testing_pipeline("CIFAR10","../../project/bests/CIFAR10_RMS/ckpt.pth")

Files already downloaded and verified
Files already downloaded and verified

The model has a validation accuracy of 82.36000%


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

The model has the following perfomances:
-	Accuracy on the standard test set: 81.48000%
-	Accuracy on the noisy test set: 56.14000%
-	Accuracy on the affine transformed test set: 50.08000%
-	Accuracy on the flipped test set: 60.50000%


##### CIFAR100

In [19]:
base_acc, noise_acc, affine_acc, flip_acc = testing_pipeline("CIFAR100","../../project/bests/CIFAR100_RMS/ckpt.pth")

Files already downloaded and verified
Files already downloaded and verified

The model has a validation accuracy of 41.72000%


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

The model has the following perfomances:
-	Accuracy on the standard test set: 39.78000%
-	Accuracy on the noisy test set: 32.07000%
-	Accuracy on the affine transformed test set: 14.96000%
-	Accuracy on the flipped test set: 26.54000%


##### FashionMNIST

In [20]:
base_acc, noise_acc, affine_acc, flip_acc = testing_pipeline("FashionMNIST","../../project/bests/FashionMNIST_RMS/ckpt.pth")


The model has a validation accuracy of 92.95000%


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

The model has the following perfomances:
-	Accuracy on the standard test set: 92.22000%
-	Accuracy on the noisy test set: 83.52000%
-	Accuracy on the affine transformed test set: 52.63000%
-	Accuracy on the flipped test set: 54.21000%


##### MNIST

In [21]:
base_acc, noise_acc, affine_acc, flip_acc = testing_pipeline("MNIST","../../project/bests/MNIST_RMS/ckpt.pth")


The model has a validation accuracy of 99.45000%


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

The model has the following perfomances:
-	Accuracy on the standard test set: 99.52000%
-	Accuracy on the noisy test set: 98.77000%
-	Accuracy on the affine transformed test set: 75.12000%
-	Accuracy on the flipped test set: 56.99000%
